In [62]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

# Загрузка данных

In [2]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [10]:
data = pd.read_excel('data_ford_price.xlsx') 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [11]:
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


## Предобработка данных

In [12]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

# выделение целевого признака
y = data['price']
x = data.drop(columns='price')

# разбиение на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

# Отбор признаков

## 1. Метод рекурсивного исключения признаков (RFE)

In [60]:
# Выбираем признаки для модели с помощью RFE

estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
columns_rfe = selector.get_feature_names_out()
print(f"Выбранные столбцы: {columns_rfe}")

Выбранные столбцы: ['year' 'cylinders' 'lat']


## 2. Метод выбора признаков на основе фильтров

In [21]:
data

,price,year,cylinders,odometer,lat,long,weather
0,43900,2016,6,43500,36.471500,-82.483400,59.0
1,15490,2009,8,98131,40.468826,-74.281734,52.0
2,2495,2002,8,201803,42.477134,-82.949564,45.0
3,1300,2000,8,170305,40.764373,-82.349503,49.0
5,6995,2003,8,167662,45.518031,-122.578752,50.0
...,...,...,...,...,...,...,...
7012,22500,2015,6,23500,32.680700,-117.169800,59.0
7013,5975,2005,8,0,38.213303,-85.785762,50.0
7014,9999,2006,8,161514,37.609783,-120.995406,59.0
7015,10900,2011,8,164000,43.140600,-93.385000,47.0


In [61]:
# Поскольку входящие и выходящий признаки числовые, выбираем признаки для модели с помощью коэффициента корреляции Пирсона

selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
columns_kbest = selector.get_feature_names_out()
print(f"Выбранные столбцы: {columns_kbest}")

Выбранные столбцы: ['year' 'cylinders' 'lat']


# Обучение модели

In [57]:
def print_metrics(y_test, y_test_predict):
    print('Test R^2: {:.3f}'.format(metrics.r2_score(y_test, y_test_predict)))
    print('Test MAE: {:.3f}'.format(metrics.mean_absolute_error(y_test, y_test_predict)))
    
    
def train_model_and_show_metrics(X, y, model = None) -> None:
    """Тренирует модель на предоставленных данных и возвращает MAE
    
    Args:
        X:  матрица признаков
        y: целевой вектор
        model: модель обучения (по-умолчанию LinearRegression)
        
    Returns:
        None
    """
    if not model:
        model = LinearRegression()
        
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model.fit(X_train, y_train)
    y_predicted = model.predict(X_test)
    
    print_metrics(y_test, y_predicted)
    
    

## 1. На признаках, найденных с помощью RFE

In [58]:
y = data['price']
x = data[columns_rfe]

train_model_and_show_metrics(x, y)


Test R^2: 0.574
Test MAE: 5206.677


## 2. На признаках, найденных с помощью метода на основе фильтров

In [59]:
y = data['price']
x = data[columns_kbest]

train_model_and_show_metrics(x, y)

Test R^2: 0.593
Test MAE: 4800.916


# Выводы

* Метод выбора признаков на основе фильтров показал лучший результат (MAE: 4800.916) по сравнению с RFE (MAE: 5206.677)
* Метрика $R^2$ меньше 0.6 для обеих моделей говорит о том, что модели недоубучены, то есть выбранных признаков недостаточно для предсказания цены автомобилей

